In [1]:
import pandas as pd
import numpy as np

df = pd.read_pickle("../../data/interim/rating.pkl")
df

,user_id,item_id,parent_asin,rating,timestamp
0,0,556,B00006HVLW,2.0,1487119257000
0,0,843,B00009R6NZ,2.0,1421898485000
0,0,1039,B0000BZL05,5.0,1479153808000
0,0,3865,B000SAKX90,5.0,1428545901000
0,0,4646,B00139XOCI,3.0,1400166889000
...,...,...,...,...,...
1999,1999,94625,B0BZZ6LR4F,5.0,1673978794550
1999,1999,95168,B0C2VL26VV,4.0,1611838236946
1999,1999,96036,B0C5C9S5BF,4.0,1542761097586
1999,1999,96165,B0C5MTHGZG,5.0,1644327808915


In [2]:
LEN_ITEM = len(df["item_id"].unique())

In [3]:
df = df.explode(["user_id", "rating", "timestamp"])
df

,user_id,item_id,parent_asin,rating,timestamp
0,0,556,B00006HVLW,2.0,1487119257000
0,0,843,B00009R6NZ,2.0,1421898485000
0,0,1039,B0000BZL05,5.0,1479153808000
0,0,3865,B000SAKX90,5.0,1428545901000
0,0,4646,B00139XOCI,3.0,1400166889000
...,...,...,...,...,...
1999,1999,94625,B0BZZ6LR4F,5.0,1673978794550
1999,1999,95168,B0C2VL26VV,4.0,1611838236946
1999,1999,96036,B0C5C9S5BF,4.0,1542761097586
1999,1999,96165,B0C5MTHGZG,5.0,1644327808915


In [4]:
df = df.groupby("user_id").agg({
    "item_id": lambda x: x.tolist(),
    "rating": lambda x: x.tolist(),
}).reset_index(drop=False)

df["item_len"] = df["item_id"].apply(lambda x: len(x))

df

,user_id,item_id,rating,item_len
0,0,"[556, 843, 1039, 3865, 4646, 5132, 5314, 5780,...","[2.0, 2.0, 5.0, 5.0, 3.0, 5.0, 5.0, 5.0, 5.0, ...",93
1,1,"[10066, 22709, 24028, 25615, 31322, 35768, 358...","[3.0, 4.0, 5.0, 5.0, 5.0, 3.0, 5.0, 4.0, 5.0, ...",96
2,2,"[41088, 41873, 46133, 54237, 57063, 58062, 593...","[1.0, 5.0, 1.0, 5.0, 5.0, 1.0, 1.0, 5.0, 5.0, ...",123
3,3,"[208, 240, 944, 1753, 2452, 3606, 3998, 4219, ...","[4.0, 5.0, 5.0, 5.0, 3.0, 4.0, 5.0, 4.0, 5.0, ...",129
4,4,"[53, 297, 308, 1661, 1773, 1805, 1884, 2368, 2...","[5.0, 5.0, 5.0, 4.0, 5.0, 4.0, 5.0, 4.0, 4.0, ...",97
...,...,...,...,...
1995,1995,"[58, 280, 333, 414, 443, 652, 830, 862, 865, 9...","[5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, ...",97
1996,1996,"[1477, 2663, 3268, 3647, 3929, 4010, 4978, 718...","[5.0, 5.0, 5.0, 1.0, 4.0, 5.0, 4.0, 5.0, 5.0, ...",93
1997,1997,"[17314, 20032, 25808, 26938, 33278, 36803, 462...","[5.0, 4.0, 3.0, 5.0, 4.0, 1.0, 5.0, 5.0, 3.0, ...",158
1998,1998,"[355, 921, 2600, 2601, 2983, 3093, 4663, 5156,...","[3.0, 5.0, 4.0, 3.0, 5.0, 5.0, 4.0, 5.0, 5.0, ...",193


In [5]:
from tqdm import tqdm
import numpy as np
from copy import deepcopy

total_item_id = np.arange(LEN_ITEM)

for idx, row in tqdm(df.iterrows()):

    neg_item_idx = np.random.choice(total_item_id, size=len(row["item_id"]), replace=False)
    while(np.intersect1d(neg_item_idx, row["item_id"]).shape[0] > 0):
        neg_item_idx = np.random.choice(total_item_id, size=len(row["item_id"]), replace=False)
    # neg_item_idx = total_item_id[~np.isin(total_item_id, row["item_id"])]
    # neg_sampled_idx = np.random.choice([i for i in range(len(neg_item_idx))], size=len(row["item_id"]), replace=False)

    item_id = row["item_id"] + neg_item_idx.tolist()
    rating = row["rating"] + [0]*len(neg_item_idx)

    df.at[idx, "item_id"] = item_id
    df.at[idx, "rating"] = rating

df

2000it [00:06, 297.73it/s]


,user_id,item_id,rating,item_len
0,0,"[556, 843, 1039, 3865, 4646, 5132, 5314, 5780,...","[2.0, 2.0, 5.0, 5.0, 3.0, 5.0, 5.0, 5.0, 5.0, ...",93
1,1,"[10066, 22709, 24028, 25615, 31322, 35768, 358...","[3.0, 4.0, 5.0, 5.0, 5.0, 3.0, 5.0, 4.0, 5.0, ...",96
2,2,"[41088, 41873, 46133, 54237, 57063, 58062, 593...","[1.0, 5.0, 1.0, 5.0, 5.0, 1.0, 1.0, 5.0, 5.0, ...",123
3,3,"[208, 240, 944, 1753, 2452, 3606, 3998, 4219, ...","[4.0, 5.0, 5.0, 5.0, 3.0, 4.0, 5.0, 4.0, 5.0, ...",129
4,4,"[53, 297, 308, 1661, 1773, 1805, 1884, 2368, 2...","[5.0, 5.0, 5.0, 4.0, 5.0, 4.0, 5.0, 4.0, 4.0, ...",97
...,...,...,...,...
1995,1995,"[58, 280, 333, 414, 443, 652, 830, 862, 865, 9...","[5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, ...",97
1996,1996,"[1477, 2663, 3268, 3647, 3929, 4010, 4978, 718...","[5.0, 5.0, 5.0, 1.0, 4.0, 5.0, 4.0, 5.0, 5.0, ...",93
1997,1997,"[17314, 20032, 25808, 26938, 33278, 36803, 462...","[5.0, 4.0, 3.0, 5.0, 4.0, 1.0, 5.0, 5.0, 3.0, ...",158
1998,1998,"[355, 921, 2600, 2601, 2983, 3093, 4663, 5156,...","[3.0, 5.0, 4.0, 3.0, 5.0, 5.0, 4.0, 5.0, 5.0, ...",193


In [6]:
df = df.explode(["item_id", "rating"])
df

,user_id,item_id,rating,item_len
0,0,556,2.0,93
0,0,843,2.0,93
0,0,1039,5.0,93
0,0,3865,5.0,93
0,0,4646,3.0,93
...,...,...,...,...
1999,1999,72419,0,106
1999,1999,82072,0,106
1999,1999,7212,0,106
1999,1999,19116,0,106


In [7]:
df.to_pickle("../../data/processed/rating_engage.pkl")